# Project: Summarization App Using LangChain and OpenAI
This is part of my **"Learn LangChain, Pinecone & OpenAI: Build Next-Gen LLM Apps"** course.

https://www.udemy.com/course/master-langchain-pinecone-openai-build-llm-applications/?referralCode=4B17E3BD4CBBEA3B8321

In [1]:
pip install -q "openai<1"

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q "langchain<1"

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [7]:
import warnings
warnings.filterwarnings('ignore')

### A) Basic Prompt

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [9]:
text= """
Mojo combines the usability of Python with the performance of C, unlocking unparalleled programmability \
of AI hardware and extensibility of AI models.
Mojo is a new programming language that bridges the gap between research and production \ 
by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. \
But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, \
we realized that programming across the entire stack was too complicated. Plus, we were writing a \
lot of MLIR by hand and not having a good time.
And although accelerators are important, one of the most prevalent and sometimes overlooked "accelerators" \
is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration \
units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, \
such as data loading, pre- and post-processing, and integrations with foreign systems. \
"""

messages = [
    SystemMessage(content='You are an expert copywriter with expertize in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text:\n TEXT: {text}')
]

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')



In [10]:
llm.get_num_tokens(text)

231

In [11]:
summary_output = llm(messages)

In [12]:
print(summary_output.content)

Mojo is a new programming language that combines the usability of Python with the performance of C. It aims to bridge the gap between research and production in the field of AI by offering portable code that is faster than C and seamlessly integrates with the Python ecosystem. Mojo was developed to simplify programming across the entire ML/AI infrastructure and to address the importance of host CPUs in AI acceleration.


### Summarizing Using Prompt Templates

In [13]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [14]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to {language}.
'''
prompt = PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [15]:
llm.get_num_tokens(promp.format(text=text, language='English'))

NameError: name 'promp' is not defined

In [16]:
chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.run({'text': text, 'language':'hindi'})

In [17]:
print(summary)

Mojo एक नया प्रोग्रामिंग भाषा है जो पायथन की उपयोगिता को सी की प्रदर्शन के साथ मिलाकर एआई हार्डवेयर की अद्वितीय प्रोग्रामबिलिटी और एआई मॉडल की विस्तारयोग्यता को खोलता है। Mojo एक नई प्रोग्रामिंग भाषा है जो शोध और उत्पादन के बीच की अंतर को पायथन की श्रेणी के साथ सिस्टम प्रोग्रामिंग और मेटाप्रोग्रामिंग के साथ मिलाती है। Mojo के साथ, आप C से तेज़ पोर्टेबल कोड लिख सकते हैं और पायथन इकोसिस्टम के साथ सहजता से इंटरऑप कर सकते हैं।


### Summarizing using SuffDocumentChain

In [18]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document


In [20]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

# text

docs = [Document(page_content=text)]
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [21]:
template = '''Write a concise and short summary of the following text.
TEXT: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [22]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.run(docs)

In [23]:
print(output_summary)

The speaker, Steve Jobs, shares three stories from his life during a commencement speech. The first story is about dropping out of college and how it led him to take a calligraphy class, which later influenced the design of the Macintosh computer. The second story is about getting fired from Apple and how it allowed him to start over and eventually create successful companies like NeXT and Pixar. The third story is about facing death when he was diagnosed with cancer and how it made him realize the importance of following his heart and not wasting time. He ends the speech by encouraging the graduates to stay hungry and stay foolish.


### Summarizing Large Documents Using map_reduce

In [24]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [25]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [26]:
llm.get_num_tokens(text)

2653

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

In [28]:
len(chunks)

2

In [29]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
output_summary = chain.run(chunks)

In [30]:
print(output_summary)

Steve Jobs shares three stories from his life, including dropping out of college and how it influenced the design of the Macintosh computer, getting fired from Apple and starting new ventures, and his experience with cancer. He emphasizes the importance of following one's passion and living each day to the fullest. The speaker encourages the audience to live their own lives, follow their hearts, and embrace qualities of staying hungry and foolish.


In [31]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [32]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

### map_reduce wich Custom Prompts

In [33]:
map_prompt = '''
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [34]:
combine_prompt = '''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=['text'])

In [35]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=False
)
output = summary_chain.run(chunks)

In [36]:
print(output)

Title: Steve Jobs' Speech on Life Lessons and Following Your Passion

Introduction:
In this speech, Steve Jobs shares three stories from his life, highlighting the importance of following one's passion and not settling for anything less. He discusses how his experiences with dropping out of college, getting fired from Apple, and battling cancer shaped his perspective on life.

Key Points:
- Story 1: Jobs dropped out of college and took a calligraphy class, which later influenced the design of the Macintosh computer.
- Story 2: Getting fired from Apple led Jobs to start new ventures and find success.
- Story 3: Jobs' experience with cancer reminded him of the importance of living each day to the fullest.
- Jobs encourages the audience to follow their passions and not be influenced by others' opinions.
- The text discusses the inevitability of death and emphasizes the importance of living one's own life.
- Following one's heart and intuition is emphasized.
- The text mentions The Whole E

### Summarizing Using the refine Chain

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader

In [38]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [39]:
pip install unstructured -q

Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install pdf2image

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [41]:
loader = UnstructuredPDFLoader('files/attention_is_all_you_need.pdf')
data = loader.load()

[nltk_data] Downloading package punkt to /home/dda/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dda/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [58]:
# print(data[0].page_content)

In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [43]:
len(chunks)

5

In [44]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [45]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('gpt-3.5-turbo')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.002:.6f}')
    
    
print_embedding_cost(chunks)

Total Tokens: 10122
Embedding Cost in USD: 0.020244


In [49]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary = chain.run(chunks)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"7

1

0

2

c

e

D

6

]

L

C

.

s

c

[

5

v

2

6

7

3

0

.

6

0

7

1

:

v

i

X

r

a

Attention Is All You Need

Ashish Vaswani∗

Google Brain

avaswani@google.com

Noam Shazeer∗

Google Brain

noam@google.com

Niki Parmar∗

Google Research

nikip@google.com

Jakob Uszkoreit∗

Google Research

usz@google.com

Llion Jones∗

Google Research

llion@google.com

Aidan N. Gomez∗ †

University of Toronto

aidan@cs.toronto.edu

Łukasz Kaiser∗

Google Brain

lukaszkaiser@google.com

Illia Polosukhin∗ ‡

illia.polosukhin@gmail.com

Abstract

The dominant sequence transduction models are based on complex recurrent or

convolutional neural networks that include an encoder and a decoder. The best

performing models also connect the encoder and decoder through an attention

mechanism. We propose a new simple network architecture, the Tra


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: The paper introduces a new network architecture called the Transformer, which is based solely on attention mechanisms and does not use recurrent or convolutional neural networks. The Transformer model achieves superior results in machine translation tasks, is more parallelizable, and requires less training time compared to existing models. The paper also discusses the advantages of self-attention and describes the architecture of the encoder and decoder stacks in the Transformer model.
We have the opportunity to refine the existing summary (only if needed) with some more context below.
------------
The two most commonly used attention functions are additive attention [2], and dot-product (multi-

plicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor

of

. Add


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: The paper introduces the Transformer, a network architecture based solely on attention mechanisms. It achieves superior results in machine translation tasks, is highly parallelizable, and requires less training time compared to existing models. The paper also discusses the advantages of self-attention and describes the architecture of the encoder and decoder stacks in the Transformer model. Additionally, it compares self-attention to recurrent and convolutional layers in terms of computational complexity, parallelizability, and the ability to learn long-range dependencies. The paper concludes that self-attention is faster and more efficient for smaller sequence lengths, and suggests the possibility of restricting self-attention to a neighborhood of input positions to improve performance with longer sequences.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: The paper introduces the Transformer, a network architecture based solely on attention mechanisms, which achieves superior results in machine translation tasks. It is highly parallelizable and requires less training time compared to existing models. The paper discusses the advantages of self-attention and describes the architecture of the encoder and decoder stacks in the Transformer model. It compares self-attention to recurrent and convolutional layers in terms of computational complexity, parallelizability, and the ability to learn long-range dependencies. The paper concludes that self-attention is faster and more efficient for smaller sequence lengths and suggests the possibility of restricting self-attention to a neighborhood of input positions to improve performance with longer sequences. The paper also 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: The paper introduces the Transformer, a network architecture based solely on attention mechanisms, which achieves superior results in machine translation tasks. It is highly parallelizable and requires less training time compared to existing models. The paper discusses the advantages of self-attention and describes the architecture of the encoder and decoder stacks in the Transformer model. It compares self-attention to recurrent and convolutional layers in terms of computational complexity, parallelizability, and the ability to learn long-range dependencies. The paper concludes that self-attention is faster and more efficient for smaller sequence lengths and suggests the possibility of restricting self-attention to a neighborhood of input positions to improve performance with longer sequences. The paper also 

In [50]:
print(output_summary)

The original summary remains unchanged as the new context does not provide any additional information that would require a refinement of the summary.


### refine With Custom Prompts

In [51]:
prompt_template = """Write a concise summary of the following extracting the key information:
Text: `{text}`
CONCISE SUMMARY:"""
initial_prompt = PromptTemplate(template=prompt_template, input_variables=['text'])

refine_template = '''
    Your job is to produce a final summary.
    I have provided an existing summary up to a certain point: {existing_answer}.
    Please refine the existing summary with some more context below.
    ------------
    {text}
    ------------
    Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
    by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
    
'''
refine_prompt = PromptTemplate(
    template=refine_template,
    input_variables=['existing_answer', 'text']
)


In [52]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=False
    
)
output_summary = chain.run(chunks)

In [53]:
print(output_summary)

INTRODUCTION:
The Transformer is a network architecture that relies solely on attention mechanisms and has several advantages over existing models. It has superior performance in machine translation tasks, increased parallelizability, and reduced training time. The architecture consists of encoder and decoder stacks with self-attention and fully connected networks. The attention mechanism used is called Scaled Dot-Product Attention. Self-attention layers have a constant number of sequentially executed operations, making them faster than recurrent layers for smaller sequence lengths. The Transformer model has been trained on the WMT 2014 English-German and English-French datasets, achieving state-of-the-art results in machine translation tasks. It has also been tested on English constituency parsing, outperforming previous models. The Transformer architecture based on attention mechanisms offers significant advantages and has the potential to improve various sequence transduction tasks.

### Summarizing Using LangChain Agents

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [3]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
wikipedia = WikipediaAPIWrapper()

In [4]:
tools = [
    Tool(
        name="Wikipedia", 
        func=wikipedia.run,
        description="Useful for when you need to get information from wikipedia about a single topic"
    )
]

In [5]:
agent_executor = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

In [6]:
output = agent_executor.run('Can you please provide a short summary of George Washington?')



> Entering new AgentExecutor chain...
I should use Wikipedia to get a brief summary of George Washington's life and accomplishments.
Action: Wikipedia
Action Input: "George Washington"
Observation: Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County, Virginia. He subsequently received his first military 

In [7]:
print(output)

George Washington was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. He led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. He was also a slave owner who had a complicated relationship with slavery. George Washington Carver was an American agricultural scientist and inventor who promoted
